In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.


In [2]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

/home/renyi/anaconda3/envs/openvoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/mnt/d/repo/OpenVoice/openvoice/api.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weigh

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []


/home/renyi/anaconda3/envs/openvoice/lib/python3.9/site-packages/wavmark/__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(resume_path, m

Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [3]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

/tmp/ipykernel_2164794/1020549311.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)


In [4]:
def openvoice_syn(reference_speaker, output_dir, texts):
    
    target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)
    src_path = f'./tmp.wav'
    base_speaker_tts.tts(texts, None, speaker='default', language='English', speed=1.0)

    encode_message = "@MyShell"
    tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=output_dir,
    message=encode_message)


In [5]:
text = "Though wise men at their end know dark is right, Because their words had forked no lightning they Do not go gentle into that good night."

In [6]:
def synthesize(input_folder, output_folder, mode):
    # open a subdir named "mode"
    os.makedirs(os.path.join(output_folder, mode), exist_ok=True)
    
    # read all files in the input folder recursively
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if mode in file:
                print(f"Processing {file}")
                current_file = os.path.join(root, file)
                target_file = os.path.join(output_folder, mode, file.replace(".wav", "_openvoice.wav"))
                openvoice_syn(current_file, target_file, text)

In [ ]:
input_folder = "/mnt/d/voicedata/Libri_adv/sampled_pair"
output_folder = "/mnt/d/voicedata/Libri_adv/openvoice"
mode = "advspeech"
synthesize(input_folder, output_folder, mode)

Processing 1272_1_advspeech_0.wav
OpenVoice version: v1
[(0.0, 5.5100625)]
after vad: dur = 5.51
 > Text splitted to sentences.
Though wise men at their end know dark is right, Because their words had forked no lightning they Do not go gentle into that good night.
 > ===========================
ðoʊ waɪz mɛn æt ðɛɹ ɛnd noʊ dɑɹk ɪz ɹaɪt, bɪˈkəz ðɛɹ wəɹdz hæd fɔɹkt noʊ ˈlaɪtnɪŋ ðeɪ du nɑt goʊ ˈdʒɛnəɫ ˈɪntu ðət gʊd naɪt.
 length:124
 length:124
Processing 1272_2_advspeech_0.wav
OpenVoice version: v1
[(0.0, 4.2300625)]
after vad: dur = 4.23
 > Text splitted to sentences.
Though wise men at their end know dark is right, Because their words had forked no lightning they Do not go gentle into that good night.
 > ===========================
ðoʊ waɪz mɛn æt ðɛɹ ɛnd noʊ dɑɹk ɪz ɹaɪt, bɪˈkəz ðɛɹ wəɹdz hæd fɔɹkt noʊ ˈlaɪtnɪŋ ðeɪ du nɑt goʊ ˈdʒɛnəɫ ˈɪntu ðət gʊd naɪt.
 length:124
 length:124
Processing 1462_1_advspeech_0.wav
OpenVoice version: v1
[(0.0, 5.82)]
after vad: dur = 5.82
 > Text splitted t

OpenVoice version: v1
[(0.0, 5.6)]
after vad: dur = 5.6
 > Text splitted to sentences.
Though wise men at their end know dark is right, Because their words had forked no lightning they Do not go gentle into that good night.
 > ===========================
ðoʊ waɪz mɛn æt ðɛɹ ɛnd noʊ dɑɹk ɪz ɹaɪt, bɪˈkəz ðɛɹ wəɹdz hæd fɔɹkt noʊ ˈlaɪtnɪŋ ðeɪ du nɑt goʊ ˈdʒɛnəɫ ˈɪntu ðət gʊd naɪt.
 length:124
 length:124
